In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score, f1_score
import torchvision
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
classes = {}
pt = 0
for i in pd.read_csv('./csv/train.csv')['label'].unique():
    ar = [0,0]
    ar[pt] = 1
    pt+=1
    classes[i] = ar

In [3]:
def get_onehot(label):
    return classes[label]

In [4]:
class SupervisedDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        label = self.df.iloc[idx]['label']
        img_name = os.path.join(self.root_dir,label,
                                str(self.df.iloc[idx]['image']))
        image = Image.open(img_name)
        onehot = np.array(get_onehot(label))
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'label': torch.Tensor(onehot).to('cuda')}

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [27]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(1000, 2)
        
    def forward(self, x):
        x = self.fc(x)
        return x
mymodel = MyModel()
resnext =  torchvision.models.resnext50_32x4d(pretrained=False, progress=True).cuda()
model = nn.Sequential(
    resnext,
    mymodel,
#     nn.Softmax(1)
)


In [31]:
model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False) #here 4 indicates 4-channel input

In [33]:
model

Sequential(
  (0): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [34]:
df_train = pd.read_csv('./csv/train.csv')
df_train = df_train.sample(frac=1)
traindataset = SupervisedDataset(df_train,root_dir='./x-ray/train/',
                                        transform=
                                 transforms.Compose([transforms.Resize((512,512)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5), (0.5))]))
trainloader = DataLoader(traindataset, batch_size=2,shuffle=True)

df_test = pd.read_csv('./csv/test.csv')
df_test = df_test.sample(frac=1)
testdataset = SupervisedDataset(df_test,root_dir='./x-ray/test/',
                                        transform=
                                transforms.Compose([transforms.Resize((512,512)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize((0.5), (0.5))]))
testloader = DataLoader(testdataset, batch_size=2, shuffle=True)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [36]:
train_f1 = []
dev_f1 = []
num_of_epochs=30
device = 'cuda'
y_pred = []
y_true = []
for epoch in range(num_of_epochs):
    i_batch = 0
    print("epoch:",epoch)
    for batch in tqdm_notebook(trainloader):
        i_batch +=1
#         print(batch['image'].shape)
#         break
#     break
        output = model(batch['image'].float().to(device))
        loss = criterion(output,batch['label'].argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy()) 
    print("training acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    train_f1.append(f1)
    print("training f1_score:", f1)
    
    y_pred = []
    y_true = []
    for batch in testloader:
        output = model(batch['image'].float().to(device))
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy())
    print("dev acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    dev_f1.append(f1)
    print("training f1_score:", f1)

epoch: 0


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[2, 1, 512, 512] to have 3 channels, but got 1 channels instead

In [27]:
df_train.iloc[0]['image']

'person1308_bacteria_3290.jpeg'